# Lab 2.3.6: GPT Text Generation

**Module:** 2.3 - Natural Language Processing & Transformers  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Understand autoregressive language modeling
- [ ] Load and use GPT-2 for text generation
- [ ] Implement greedy decoding from scratch
- [ ] Implement temperature sampling
- [ ] Implement top-k and top-p (nucleus) sampling
- [ ] Implement beam search
- [ ] Compare different decoding strategies

---

## Prerequisites

- Completed: Labs 2.3.1-2.3.5
- Knowledge of: Probability distributions, sampling

---

## Real-World Context

**Text generation powers:**
- **ChatGPT, Claude** - Conversational AI
- **GitHub Copilot** - Code generation
- **Jasper, Copy.ai** - Marketing copy
- **AI Dungeon** - Interactive storytelling

The decoding strategy dramatically affects output quality:
- Greedy: Fast but repetitive
- Sampling: Creative but can be random
- Beam search: High quality but less diverse
- Top-p: Good balance of quality and creativity

---

## ELI5: How GPT Generates Text

> **Imagine a very good game of "complete the sentence."**
>
> You start with: "The cat sat on the..."
> GPT thinks: "What word usually comes next?"
> - "mat" - 40% likely
> - "chair" - 25% likely
> - "floor" - 20% likely
> - "sofa" - 10% likely
> - ... (thousands of options)
>
> **Different strategies for picking the next word:**
> - **Greedy**: Always pick "mat" (highest probability)
> - **Sampling**: Randomly pick based on probabilities (might get "sofa")
> - **Top-k**: Only consider top 50 options, then sample
> - **Top-p**: Only consider options until you have 90% probability mass
>
> Then repeat: "The cat sat on the mat..." → what's next?

### Key Insight: Autoregressive Generation

GPT generates one token at a time, feeding its output back as input:

```
Input: "The cat"     → Output: "sat"
Input: "The cat sat" → Output: "on"
Input: "The cat sat on" → Output: "the"
...
```

---

## Part 1: Setup and Load GPT-2

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import time

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

torch.manual_seed(42)

In [ ]:
# Load GPT-2 model and tokenizer
print("Loading GPT-2...")

# Use GPT-2 Medium for better quality (on DGX Spark, we can use larger models!)
model_name = "gpt2"  # or "gpt2-medium", "gpt2-large", "gpt2-xl"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model = model.to(device)
model.eval()  # Set to evaluation mode

# Set pad token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

print(f"\nModel: {model_name}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Vocabulary size: {tokenizer.vocab_size:,}")

In [ ]:
# Quick test: get next token probabilities

prompt = "The capital of France is"

# Tokenize
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
print(f"Prompt: '{prompt}'")
print(f"Token IDs: {input_ids[0].tolist()}")
print(f"Tokens: {tokenizer.convert_ids_to_tokens(input_ids[0])}")

# Get model output
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits  # (batch, seq_len, vocab_size)

# Get probabilities for next token
next_token_logits = logits[0, -1, :]  # Last position
probs = F.softmax(next_token_logits, dim=-1)

# Top 10 most likely next tokens
top_probs, top_indices = probs.topk(10)

print(f"\nTop 10 most likely next tokens:")
for prob, idx in zip(top_probs, top_indices):
    token = tokenizer.decode([idx])
    print(f"  '{token}': {prob.item()*100:.1f}%")

---

## Part 2: Greedy Decoding

The simplest strategy: always pick the most likely next token.

In [ ]:
def greedy_decode(model, tokenizer, prompt, max_length=50, device='cpu'):
    """
    Generate text using greedy decoding.
    
    At each step, pick the token with highest probability.
    
    Args:
        model: GPT-2 model
        tokenizer: GPT-2 tokenizer
        prompt: Starting text
        max_length: Maximum tokens to generate
        device: Device to run on
        
    Returns:
        Generated text
    """
    model.eval()
    
    # Tokenize prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    generated = input_ids[0].tolist()
    
    for _ in range(max_length):
        # Get model predictions
        with torch.no_grad():
            outputs = model(torch.tensor([generated]).to(device))
            logits = outputs.logits[0, -1, :]  # Last token's predictions
        
        # Greedy: pick highest probability token
        next_token = logits.argmax().item()
        
        # Stop if we hit end-of-sequence
        if next_token == tokenizer.eos_token_id:
            break
            
        generated.append(next_token)
    
    return tokenizer.decode(generated)

# Test greedy decoding
prompt = "Artificial intelligence will"
result = greedy_decode(model, tokenizer, prompt, max_length=30, device=device)

print("Greedy Decoding:")
print(f"Prompt: '{prompt}'")
print(f"Output: '{result}'")

In [ ]:
# Problem with greedy: repetition!

print("Greedy decoding problem - repetition:")
print("=" * 60)

prompts = [
    "Once upon a time",
    "The meaning of life is",
    "In a galaxy far far away",
]

for prompt in prompts:
    result = greedy_decode(model, tokenizer, prompt, max_length=50, device=device)
    print(f"\nPrompt: '{prompt}'")
    print(f"Output: {result}")

print("\n⚠️ Notice the repetition in the outputs!")

---

## Part 3: Temperature Sampling

### ELI5: What is Temperature?

> **Temperature controls "confidence" in the distribution.**
>
> - **Temperature = 1.0**: Use probabilities as-is
> - **Temperature < 1.0**: More confident (sharper distribution, more likely to pick top token)
> - **Temperature > 1.0**: Less confident (flatter distribution, more random)

Mathematically: `probs = softmax(logits / temperature)`

In [ ]:
# Visualize temperature effect

logits = torch.tensor([5.0, 3.0, 2.0, 1.0, 0.5])

temperatures = [0.1, 0.5, 1.0, 1.5, 2.0]

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for ax, temp in zip(axes, temperatures):
    probs = F.softmax(logits / temp, dim=-1)
    ax.bar(range(5), probs.numpy())
    ax.set_title(f'Temp = {temp}')
    ax.set_ylim(0, 1)
    ax.set_xlabel('Token')
    if temp == 0.1:
        ax.set_ylabel('Probability')

plt.suptitle('Effect of Temperature on Probability Distribution', y=1.05)
plt.tight_layout()
plt.show()

print("Lower temperature → More deterministic (like greedy)")
print("Higher temperature → More random (more creative but can be nonsense)")

In [ ]:
def sample_with_temperature(model, tokenizer, prompt, max_length=50, 
                           temperature=1.0, device='cpu'):
    """
    Generate text using temperature sampling.
    
    Args:
        temperature: Controls randomness (lower = more deterministic)
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    generated = input_ids[0].tolist()
    
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(torch.tensor([generated]).to(device))
            logits = outputs.logits[0, -1, :]
        
        # Apply temperature
        scaled_logits = logits / temperature
        probs = F.softmax(scaled_logits, dim=-1)
        
        # Sample from distribution
        next_token = torch.multinomial(probs, num_samples=1).item()
        
        if next_token == tokenizer.eos_token_id:
            break
            
        generated.append(next_token)
    
    return tokenizer.decode(generated)

# Compare different temperatures
prompt = "The scientist discovered a"

print(f"Prompt: '{prompt}'")
print("=" * 60)

for temp in [0.3, 0.7, 1.0, 1.5]:
    result = sample_with_temperature(
        model, tokenizer, prompt, 
        max_length=30, temperature=temp, device=device
    )
    print(f"\nTemp={temp}:")
    print(f"  {result}")

---

## Part 4: Top-k Sampling

Only consider the top k most likely tokens, then sample from those.

In [ ]:
def top_k_sampling(model, tokenizer, prompt, max_length=50, 
                   k=50, temperature=1.0, device='cpu'):
    """
    Generate text using top-k sampling.
    
    Only sample from the k most likely tokens.
    
    Args:
        k: Number of top tokens to consider
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    generated = input_ids[0].tolist()
    
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(torch.tensor([generated]).to(device))
            logits = outputs.logits[0, -1, :]
        
        # Apply temperature
        scaled_logits = logits / temperature
        
        # Get top k
        top_k_logits, top_k_indices = scaled_logits.topk(k)
        
        # Sample from top k
        probs = F.softmax(top_k_logits, dim=-1)
        selected_idx = torch.multinomial(probs, num_samples=1).item()
        next_token = top_k_indices[selected_idx].item()
        
        if next_token == tokenizer.eos_token_id:
            break
            
        generated.append(next_token)
    
    return tokenizer.decode(generated)

# Compare different k values
prompt = "In the year 2050, robots will"

print(f"Prompt: '{prompt}'")
print("=" * 60)

for k in [5, 20, 50, 100]:
    result = top_k_sampling(
        model, tokenizer, prompt,
        max_length=30, k=k, temperature=0.8, device=device
    )
    print(f"\nTop-k={k}:")
    print(f"  {result}")

---

## Part 5: Top-p (Nucleus) Sampling

### ELI5: Top-p Sampling

> **Instead of fixed k tokens, include tokens until you have p% of probability mass.**
>
> If top 3 tokens have 40%, 30%, 20% = 90% total:
> - With p=0.9, only consider these 3 tokens
> - With p=0.95, might include 4th token too
>
> **Advantage:** Adapts to the situation!
> - Confident prediction (one token is 95%) → only consider that one
> - Uncertain (many similar probabilities) → consider many options

In [ ]:
def top_p_sampling(model, tokenizer, prompt, max_length=50,
                   p=0.9, temperature=1.0, device='cpu'):
    """
    Generate text using nucleus (top-p) sampling.
    
    Include smallest set of tokens whose cumulative probability >= p.
    
    Args:
        p: Cumulative probability threshold
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    generated = input_ids[0].tolist()
    
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(torch.tensor([generated]).to(device))
            logits = outputs.logits[0, -1, :]
        
        # Apply temperature
        scaled_logits = logits / temperature
        probs = F.softmax(scaled_logits, dim=-1)
        
        # Sort by probability (descending)
        sorted_probs, sorted_indices = probs.sort(descending=True)
        
        # Compute cumulative probabilities
        cumsum = sorted_probs.cumsum(dim=-1)
        
        # Find cutoff (first position where cumsum >= p)
        cutoff_idx = (cumsum >= p).nonzero()[0].item() + 1
        
        # Keep only tokens in the nucleus
        nucleus_probs = sorted_probs[:cutoff_idx]
        nucleus_indices = sorted_indices[:cutoff_idx]
        
        # Re-normalize probabilities
        nucleus_probs = nucleus_probs / nucleus_probs.sum()
        
        # Sample
        selected_idx = torch.multinomial(nucleus_probs, num_samples=1).item()
        next_token = nucleus_indices[selected_idx].item()
        
        if next_token == tokenizer.eos_token_id:
            break
            
        generated.append(next_token)
    
    return tokenizer.decode(generated)

# Compare different p values
prompt = "The future of technology is"

print(f"Prompt: '{prompt}'")
print("=" * 60)

for p in [0.5, 0.8, 0.9, 0.95]:
    result = top_p_sampling(
        model, tokenizer, prompt,
        max_length=30, p=p, temperature=0.8, device=device
    )
    print(f"\nTop-p={p}:")
    print(f"  {result}")

---

## Part 6: Beam Search

### ELI5: Beam Search

> **Instead of keeping just one sequence, keep track of multiple promising sequences.**
>
> Imagine you're navigating a maze:
> - Greedy: Take the best-looking path at each junction
> - Beam search: Keep track of top 5 paths, explore all of them
>
> This finds better overall solutions, not just locally optimal ones.

In [ ]:
def beam_search(model, tokenizer, prompt, max_length=50,
                beam_width=5, device='cpu'):
    """
    Generate text using beam search.
    
    Maintain beam_width best sequences at each step.
    
    Args:
        beam_width: Number of sequences to track
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Each beam: (sequence, log_prob)
    beams = [(input_ids[0].tolist(), 0.0)]
    completed = []
    
    for _ in range(max_length):
        candidates = []
        
        for seq, score in beams:
            with torch.no_grad():
                outputs = model(torch.tensor([seq]).to(device))
                logits = outputs.logits[0, -1, :]
            
            log_probs = F.log_softmax(logits, dim=-1)
            
            # Get top beam_width tokens
            top_log_probs, top_indices = log_probs.topk(beam_width)
            
            for log_prob, idx in zip(top_log_probs, top_indices):
                new_seq = seq + [idx.item()]
                new_score = score + log_prob.item()
                
                if idx.item() == tokenizer.eos_token_id:
                    completed.append((new_seq, new_score))
                else:
                    candidates.append((new_seq, new_score))
        
        # Keep top beam_width candidates
        candidates.sort(key=lambda x: x[1], reverse=True)
        beams = candidates[:beam_width]
        
        if not beams:
            break
    
    # Add remaining beams to completed
    completed.extend(beams)
    
    # Return best sequence (normalized by length)
    if completed:
        # Length normalize scores
        completed = [(seq, score / len(seq)) for seq, score in completed]
        best_seq, _ = max(completed, key=lambda x: x[1])
        return tokenizer.decode(best_seq)
    else:
        return tokenizer.decode(beams[0][0])

# Test beam search
prompt = "The best way to learn programming is"

print(f"Prompt: '{prompt}'")
print("=" * 60)

for beam_width in [1, 3, 5, 10]:
    result = beam_search(
        model, tokenizer, prompt,
        max_length=30, beam_width=beam_width, device=device
    )
    print(f"\nBeam width={beam_width}:")
    print(f"  {result}")

---

## Part 7: Comparison of Strategies

In [ ]:
# Compare all strategies on the same prompt

prompt = "Once upon a time in a magical kingdom, there lived a"

print(f"Prompt: '{prompt}'")
print("=" * 70)

strategies = [
    ("Greedy", lambda: greedy_decode(model, tokenizer, prompt, 40, device)),
    ("Temp=0.7", lambda: sample_with_temperature(model, tokenizer, prompt, 40, 0.7, device)),
    ("Top-k=40", lambda: top_k_sampling(model, tokenizer, prompt, 40, 40, 0.8, device)),
    ("Top-p=0.9", lambda: top_p_sampling(model, tokenizer, prompt, 40, 0.9, 0.8, device)),
    ("Beam=5", lambda: beam_search(model, tokenizer, prompt, 40, 5, device)),
]

for name, func in strategies:
    result = func()
    print(f"\n{name}:")
    print(f"  {result}")

In [ ]:
# Measure generation speed

prompt = "Artificial intelligence is"
max_length = 50
num_runs = 3

print("Generation Speed Comparison:")
print("=" * 50)

for name, func in strategies:
    times = []
    for _ in range(num_runs):
        start = time.time()
        _ = func()
        times.append(time.time() - start)
    
    avg_time = sum(times) / len(times)
    tokens_per_sec = max_length / avg_time
    print(f"  {name}: {avg_time*1000:.0f}ms ({tokens_per_sec:.1f} tokens/sec)")

---

## Part 8: Using Hugging Face's Generate Method

In production, use the optimized built-in method:

In [ ]:
# Hugging Face's generate method is much faster and more feature-rich

prompt = "The secret to happiness is"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

print(f"Prompt: '{prompt}'")
print("=" * 60)

# Greedy
outputs = model.generate(
    input_ids, 
    max_new_tokens=30,
    do_sample=False  # Greedy
)
print(f"\nGreedy: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

# Top-p sampling
outputs = model.generate(
    input_ids,
    max_new_tokens=30,
    do_sample=True,
    top_p=0.9,
    temperature=0.8
)
print(f"\nTop-p: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

# Top-k sampling
outputs = model.generate(
    input_ids,
    max_new_tokens=30,
    do_sample=True,
    top_k=50,
    temperature=0.8
)
print(f"\nTop-k: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

# Beam search
outputs = model.generate(
    input_ids,
    max_new_tokens=30,
    num_beams=5,
    no_repeat_ngram_size=2  # Prevent repetition
)
print(f"\nBeam: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

---

## Part 9: Advanced: Repetition Penalty

In [ ]:
# Repetition is a common problem. Let's fix it!

def top_p_with_repetition_penalty(model, tokenizer, prompt, max_length=50,
                                  p=0.9, temperature=0.8, 
                                  repetition_penalty=1.2, device='cpu'):
    """
    Top-p sampling with repetition penalty.
    
    Args:
        repetition_penalty: Penalty for repeated tokens (>1 discourages repetition)
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    generated = input_ids[0].tolist()
    
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(torch.tensor([generated]).to(device))
            logits = outputs.logits[0, -1, :]
        
        # Apply repetition penalty
        for token_id in set(generated):
            if logits[token_id] > 0:
                logits[token_id] = logits[token_id] / repetition_penalty
            else:
                logits[token_id] = logits[token_id] * repetition_penalty
        
        # Apply temperature and top-p
        scaled_logits = logits / temperature
        probs = F.softmax(scaled_logits, dim=-1)
        
        sorted_probs, sorted_indices = probs.sort(descending=True)
        cumsum = sorted_probs.cumsum(dim=-1)
        cutoff_idx = (cumsum >= p).nonzero()[0].item() + 1
        
        nucleus_probs = sorted_probs[:cutoff_idx]
        nucleus_indices = sorted_indices[:cutoff_idx]
        nucleus_probs = nucleus_probs / nucleus_probs.sum()
        
        selected_idx = torch.multinomial(nucleus_probs, num_samples=1).item()
        next_token = nucleus_indices[selected_idx].item()
        
        if next_token == tokenizer.eos_token_id:
            break
            
        generated.append(next_token)
    
    return tokenizer.decode(generated)

# Compare with and without repetition penalty
prompt = "The weather today is very nice because the weather is"

print(f"Prompt: '{prompt}'")
print("=" * 60)

result_no_penalty = top_p_sampling(
    model, tokenizer, prompt, 40, 0.9, 0.8, device
)
print(f"\nWithout penalty:")
print(f"  {result_no_penalty}")

result_with_penalty = top_p_with_repetition_penalty(
    model, tokenizer, prompt, 40, 0.9, 0.8, 1.5, device
)
print(f"\nWith penalty (1.5):")
print(f"  {result_with_penalty}")

---

## Try It Yourself: Exercises

### Exercise 1: Implement Contrastive Search

Contrastive search balances likelihood with diversity:

```
score(token) = (1-alpha) * log_prob(token) - alpha * max_cosine_sim(token, previous_tokens)
```

In [ ]:
# YOUR CODE HERE
# Implement contrastive search decoding

### Exercise 2: Story Generator

Create an interactive story generator that:
1. Takes a story premise
2. Generates a paragraph
3. Lets user choose what happens next
4. Continues the story

In [ ]:
# YOUR CODE HERE
# Create an interactive story generator

---

## Common Mistakes

### Mistake 1: Not handling special tokens

In [ ]:
# Wrong: Not checking for EOS token
def wrong_generate(model, prompt, length):
    for _ in range(length):
        # ... generate token ...
        # Never checks if it's EOS!
        pass

# Right: Stop at EOS
def right_generate(model, tokenizer, prompt, length):
    for _ in range(length):
        # ... generate next_token ...
        if next_token == tokenizer.eos_token_id:
            break  # Stop generation!

print("Always check for EOS token to stop generation properly!")

### Mistake 2: Temperature = 0

In [ ]:
# Wrong: Temperature of 0 causes division by zero!
# logits / 0 = infinity

# If you want deterministic, use:
# 1. Very small temperature (0.01)
# 2. Greedy decoding (just take argmax)

print("Never use temperature=0! Use very small values or greedy instead.")

### Mistake 3: Not using KV cache

In [ ]:
# Wrong: Recompute all tokens at each step (slow!)
def slow_generate(model, input_ids):
    for _ in range(100):
        outputs = model(input_ids)  # Recomputes ALL tokens!
        next_token = outputs.logits[0, -1].argmax()
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0).unsqueeze(0)], dim=1)

# Right: Use KV cache (Hugging Face does this automatically)
# The generate() method uses past_key_values to avoid recomputation

print("Hugging Face's generate() uses KV cache for speed.")
print("For custom implementations, use past_key_values parameter.")

---

## Decoding Strategy Summary

| Strategy | Pros | Cons | Best For |
|----------|------|------|----------|
| Greedy | Fast, deterministic | Repetitive, boring | Quick tests |
| Temperature | Controls randomness | Can be too random | Creative writing |
| Top-k | Removes unlikely tokens | Fixed k not adaptive | General use |
| Top-p | Adaptive, high quality | Slightly slower | Production use |
| Beam | Finds optimal sequence | Less diverse, slower | Translation, summarization |

---

## Checkpoint

You've learned:
- ✅ How autoregressive generation works
- ✅ Greedy decoding and its limitations
- ✅ Temperature sampling for controlling randomness
- ✅ Top-k sampling to limit vocabulary
- ✅ Top-p (nucleus) sampling for adaptive selection
- ✅ Beam search for finding optimal sequences
- ✅ Repetition penalties to avoid boring text

---

## Challenge (Optional)

Implement **speculative decoding** for faster generation:
1. Use a small "draft" model to generate multiple tokens quickly
2. Use the large model to verify/correct them in parallel
3. Accept correct tokens, regenerate incorrect ones

This can provide 2-3x speedup!

In [ ]:
# Challenge: Implement speculative decoding
# YOUR CODE HERE

---

## Further Reading

- [The Curious Case of Neural Text Degeneration](https://arxiv.org/abs/1904.09751) - Top-p sampling paper
- [CTRL: Conditional Transformer LM](https://arxiv.org/abs/1909.05858) - Controllable generation
- [Contrastive Search](https://arxiv.org/abs/2202.06417) - Better decoding
- [Speculative Decoding](https://arxiv.org/abs/2211.17192) - Fast generation

---

## Cleanup

In [ ]:
# Clean up
import gc

del model, tokenizer

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("Memory cleared!")
print("\n" + "=" * 60)
print("🎉 Congratulations! You've completed Module 2.3!")
print("=" * 60)
print("\nYou now understand:")
print("  ✅ Attention mechanisms")
print("  ✅ Transformer architecture")
print("  ✅ Positional encodings")
print("  ✅ Tokenization")
print("  ✅ BERT fine-tuning")
print("  ✅ GPT text generation")
print("\nYou're ready for Module 2.4: Efficient Architectures!")